# Getting started

In [1]:
# do basic imports and unpack McMurdo data

from pmagpy import ipmag
reload(ipmag)
from pmagpy import pmag
from pmagpy import new_builder as nb
from programs import data_model3
import os
import pandas as pd
import numpy as np
from pandas import DataFrame
from pmagpy.new_builder import Contribution

wdir = os.path.join(os.getcwd(), "3_0", "McMurdo")
#infile = os.path.join(wdir, "lawrence09.v30.txt")
infile = os.path.join(wdir, "mcmurdo3-with-upgrade.txt") 
ipmag.download_magic(infile, overwrite=True, dir_path=wdir)


/Users/nebula/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


got full_df
got parsed_df
working on:  'contribution'
1  records written to file  /Users/nebula/Python/PmagPy/3_0/McMurdo/contribution.txt
contribution  data put in  /Users/nebula/Python/PmagPy/3_0/McMurdo/contribution.txt
working on:  'locations'
5  records written to file  /Users/nebula/Python/PmagPy/3_0/McMurdo/locations.txt
locations  data put in  /Users/nebula/Python/PmagPy/3_0/McMurdo/locations.txt
working on:  'sites'
542  records written to file  /Users/nebula/Python/PmagPy/3_0/McMurdo/sites.txt
sites  data put in  /Users/nebula/Python/PmagPy/3_0/McMurdo/sites.txt
working on:  'samples'
1418  records written to file  /Users/nebula/Python/PmagPy/3_0/McMurdo/samples.txt
samples  data put in  /Users/nebula/Python/PmagPy/3_0/McMurdo/samples.txt
working on:  'specimens'
1393  records written to file  /Users/nebula/Python/PmagPy/3_0/McMurdo/specimens.txt
specimens  data put in  /Users/nebula/Python/PmagPy/3_0/McMurdo/specimens.txt
working on:  'measurements'
25470  records written to

True

# Several ways of creating a contribution

In [2]:
reload(nb)

# test out various ways of creating a contribution

#class Contribution(object):
#    """                                                                                                                   
#    A Contribution is a collection of MagicDataFrames,                                                                    
#    each of which corresponds to one MagIC table.                                                                         
#    The Contribution object also has methods for                                                                          
#    manipulating one or more tables in the contribution --                                                                
#    for example, renaming a site.                                                                                         
#    """
#    def __init__(self, directory, read_tables='all',
#                 custom_filenames=None, single_file=None):



# make contribution reading in all default filenames from working directory
con = nb.Contribution(wdir)
print 'tables created:', con.tables.keys()
print '-'

# make contribution with some custom filenames
con = nb.Contribution(wdir, custom_filenames={'specimens': 'custom_specimens.txt'})
print 'tables created:', con.tables.keys()
print '-'

# make contribution with custom filenames, and only read in the specimen table to start
con = Contribution(wdir, read_tables=['specimens'], custom_filenames={'sites': 'custom_sites.txt',
                                                                      'specimens': 'custom_specimens.txt'})
print 'tables created:', con.tables.keys()
print '-'

# make contribution with a single, mystery file (can be any datatype)
con = nb.Contribution(wdir, single_file='sites.txt')
print 'tables created:', con.tables.keys()
print '-'


tables created: ['measurements', 'ages', 'sites', 'locations', 'samples', 'criteria', 'images', 'contribution', 'specimens']
-
tables created: ['measurements', 'ages', 'sites', 'locations', 'samples', 'criteria', 'images', 'contribution', 'specimens']
-
tables created: ['specimens']
-
tables created: ['sites']
-


In [3]:
# make McMurdo contribution, starting with specimens table

reload(nb)

con = nb.Contribution(wdir, read_tables=['specimens'], custom_filenames={'specimens': 'custom_specimens.txt', 'samples': 'custom_samples.txt',
                                                                         'sites': 'custom_sites.txt'})

print con.filenames
print con.tables.keys()

{'measurements': 'measurements.txt', 'ages': 'ages.txt', 'sites': 'custom_sites.txt', 'locations': 'locations.txt', 'samples': 'custom_samples.txt', 'criteria': 'criteria.txt', 'images': 'images.txt', 'contribution': 'contribution.txt', 'specimens': 'custom_specimens.txt'}
['specimens']


In [4]:
# then, add another table to the contribution
# here, we are providing data type but no filename
# this works because we already gave the custom sample filename when we created the contribution
# so the contribution already knows where to look (con.filenames)
con.add_magic_table('samples')
print con.tables.keys()

['specimens', 'samples']


In [5]:
# add another table to the same contribution
# this time, provide a filename but no data type

con.add_magic_table(dtype="unknown", fname="criteria.txt")
# criteria table now included
print con.tables.keys()

['specimens', 'samples', 'criteria']


# Functionality with a contribution

In [6]:
# create full McMurdo contribution

reload(nb)

con = nb.Contribution(wdir, custom_filenames={'specimens': 'specimens.txt', 'samples': 'samples.txt',
                                             'sites': 'sites.txt'})


In [7]:
con.tables['locations'].df[['sites']]

,sites
location,
McMurdo,None
McMurdo,mc03 : mc04 : mc06 : mc07 : mc08 : mc09 : mc10...
McMurdo,mc102 : mc103 : mc105 : mc109 : mc110 : mc112 ...
McMurdo,mc02 : mc03 : mc04 : mc06 : mc07 : mc08 : mc09...
McMurdo,mc09 : mc105 : mc109 : mc111 : mc113 : mc115 :...


## Rename an item

In [8]:
reload(nb)
# rename one of the Contribution's sites
con.rename_item('sites', 'mc03', 'extra_special_site')
con.tables['sites'].df.ix[['extra_special_site']]
# all rows previously named 'mc01' are now named 'extra_special_site'

can only join an iterable


,age,age_sigma,age_unit,analysts,citations,criteria,description,dir_alpha95,dir_comp_name,dir_dec,...,vadm_n_samples,vadm_sigma,vdm,vdm_n_samples,vdm_sigma,vgp_dm,vgp_dp,vgp_lat,vgp_lon,vgp_n_samples
site,,,,,,,,,,,,,,,,,,,,,
extra_special_site,None,None,None,None,This study,None,"10-m thick basalt flow, NE of Scott Base, Hut ...",None,None,None,...,None,None,None,None,None,None,None,None,None,None
extra_special_site,0.348,0.004,Ma,Lisa Tauxe,This study,DE-SPEC,Direction included in Pmag_Results.,2.3,A,352,...,None,None,None,None,None,None,None,None,None,None
extra_special_site,0.348,0.004,Ma,Kristin Lawrence,This study,DE-SITE,Site VGPA comp: (geog. coord).,2.3,None,352,...,None,None,None,None,None,3.8,4.4,87.1,123.1,6
extra_special_site,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [9]:
# additionally, 'mc03' has been replaced in the location table under site_names
#con.tables['locations'].df.ix[["Osler Volcanics, Nipigon Strait, Lower Reversed"]][['site_names']]
con.tables['locations'].df[['sites']]#, 'sites_list']]

,sites
location,
McMurdo,None
McMurdo,extra_special_site:mc04:mc06:mc07:mc08:mc09:mc...
McMurdo,mc102 : mc103 : mc105 : mc109 : mc110 : mc112 ...
McMurdo,mc02:extra_special_site:mc04:mc06:mc07:mc08:mc...
McMurdo,mc09 : mc105 : mc109 : mc111 : mc113 : mc115 :...


## Propagate data from one table into another

In [10]:

# normally, each table only has one relationship up (i.e., a measurement table will have specimen name, but not sample name)
# sometimes, you need to access location_name at the site level (for example)
# this function propagates names down through any available tables
# the code snippet below won't work if the Contribution can't access the sample and site files!


reload(nb)

con = nb.Contribution(wdir, custom_filenames={'specimens': 'custom_specimens.txt', 'samples': 'custom_samples.txt',
                                             'sites': 'custom_sites.txt'})

con.propagate_name_down('location', 'specimens')

# specimens table now has sample, site, and location_names
con.tables['specimens'].df[['specimen', 'sample', 'site', 'location']].head()


,specimen,sample,site,location
specimen,,,,
mc01a,mc01a,mc01a,mc01,McMurdo
mc01a,mc01a,mc01a,mc01,McMurdo
mc01b,mc01b,mc01b,mc01,McMurdo
mc01b,mc01b,mc01b,mc01,McMurdo
mc01c,mc01c,mc01c,mc01,McMurdo


In [11]:
# this function propagates values from arbitrary columns down
# i.e., get sample-level azimuth into the measurements table
# note: this will NOT work with names (specimen_name, sample_name, etc.).  
# for those relationships, use the above function: propagate_name_down

reload(nb)
con = nb.Contribution(wdir, custom_filenames={'specimens': 'custom_specimens.txt', 'samples': 'custom_samples.txt',
                                             'sites': 'custom_sites.txt'})

meas_container = con.tables['measurements']
meas_df = meas_container.df

meas_df = con.propagate_cols_down(['azimuth', 'dip', 'fake_col'], 'measurements', 'samples')
meas_df.head()[['azimuth', 'dip']]

-W- Column 'fake_col' isn't in samples table, skipping it


,azimuth,dip
measurement,,
mc01f-LP-DIR-AF1,94,-55
mc01f-LP-DIR-AF2,94,-55
mc01f-LP-DIR-AF3,94,-55
mc01f-LP-DIR-AF4,94,-55
mc01f-LP-DIR-AF5,94,-55


## Writing out a MagIC file

In [12]:
con = nb.Contribution(wdir, custom_filenames={'specimens': 'custom_specimens.txt', 'samples': 'custom_samples.txt',
                                             'sites': 'custom_sites.txt'})
site_container = con.tables['sites']
site_container.write_magic_file(custom_name='fancy_sites.txt', dir_path='./3_0/McMurdo')

site_container.df

-I- writing sites data to /Users/nebula/Python/PmagPy/3_0/McMurdo/fancy_sites.txt


,age,age_sigma,age_unit,analysts,citations,criteria,description,dir_alpha95,dir_comp_name,dir_dec,...,vadm_n_samples,vadm_sigma,vdm,vdm_n_samples,vdm_sigma,vgp_dm,vgp_dp,vgp_lat,vgp_lon,vgp_n_samples
site,,,,,,,,,,,,,,,,,,,,,
mc01,1.18,0.005,Ma,Lisa Tauxe,This study,IE-SPEC,"Trachyte flow, N flank Observation hill, Hut P...",None,None,None,...,None,None,None,None,None,None,None,None,None,None
mc01,1.18,0.005,Ma,Lisa Tauxe,This study,DE-SPEC,Direction included in Pmag_Results.,4.2,A,258.6,...,None,None,None,None,None,None,None,None,None,None
mc01,1.18,0.005,Ma,Kristin Lawrence,This study,DE-SITE,Site VGPA comp: (geog. coord).,4.2,None,258.6,...,None,None,None,None,None,4.5,8.1,-67.3,95.2,7
mc01,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
mc02,None,None,None,None,This study,None,"Upper basalt flow, S. flank Crater Hill, Hut P...",None,None,None,...,None,None,None,None,None,None,None,None,None,None
mc02,0.33,0.01,Ma,Lisa Tauxe,This study,DE-SPEC,Direction included in Pmag_Results.,2.1,A,328.6,...,None,None,None,None,None,None,None,None,None,None
mc02,0.33,0.01,Ma,Kristin Lawrence,This study,DE-SITE,Site VGPA comp: (geog. coord).,2.1,None,328.6,...,None,None,None,None,None,2.5,4.1,79,101.2,6
mc02,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
mc03,None,None,None,None,This study,None,"10-m thick basalt flow, NE of Scott Base, Hut ...",None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [13]:
site_container.df.index.name

'site'

## Mucking around with measurements

In [14]:
con = nb.Contribution(wdir, custom_filenames={'specimens': 'custom_specimens.txt', 'samples': 'custom_samples.txt',
                                             'sites': 'custom_sites.txt'})

meas_container = con.tables['measurements']
meas_data = meas_container.df

meas_data['treatment'] = meas_data['treat_ac_field'].where(cond=meas_data['treat_ac_field'] != "0", other=meas_data['treat_temp'])
meas_data[['treatment', 'treat_ac_field', 'treat_temp']]

meas_data['treat_ac_field'].ix[0] = None
meas_data['treat_ac_field'] = meas_data['treat_ac_field'].astype(float)

meas_data[['treatment', 'treat_ac_field', 'treat_temp']].head()


,treatment,treat_ac_field,treat_temp
measurement,,,
mc01f-LP-DIR-AF1,273,NaN,273
mc01f-LP-DIR-AF2,0.005,0.0050,273
mc01f-LP-DIR-AF3,0.0075,0.0075,273
mc01f-LP-DIR-AF4,0.01,0.0100,273
mc01f-LP-DIR-AF5,0.0125,0.0125,273


## Dealing with criteria

In [15]:
#  grab a copy of the criteria and sites table to play with
criteria = con.tables['criteria'].df.copy()
sites = con.tables['sites'].df.copy()
locations = con.tables['locations'].df.copy()
specimens = con.tables['specimens'].df.copy()
samples = con.tables['samples'].df.copy()
sites.head()

,age,age_sigma,age_unit,analysts,citations,criteria,description,dir_alpha95,dir_comp_name,dir_dec,...,vadm_n_samples,vadm_sigma,vdm,vdm_n_samples,vdm_sigma,vgp_dm,vgp_dp,vgp_lat,vgp_lon,vgp_n_samples
site,,,,,,,,,,,,,,,,,,,,,
mc01,1.18,0.005,Ma,Lisa Tauxe,This study,IE-SPEC,"Trachyte flow, N flank Observation hill, Hut P...",None,None,None,...,None,None,None,None,None,None,None,None,None,None
mc01,1.18,0.005,Ma,Lisa Tauxe,This study,DE-SPEC,Direction included in Pmag_Results.,4.2,A,258.6,...,None,None,None,None,None,None,None,None,None,None
mc01,1.18,0.005,Ma,Kristin Lawrence,This study,DE-SITE,Site VGPA comp: (geog. coord).,4.2,None,258.6,...,None,None,None,None,None,4.5,8.1,-67.3,95.2,7
mc01,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
mc02,None,None,None,None,This study,None,"Upper basalt flow, S. flank Crater Hill, Hut P...",None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [16]:
# get all criteria for sites

# only criteria with 'site' in table_column_name
cond = criteria.index.str.contains('site')
site_crit = criteria[cond].copy()
# remove table name from index
site_crit.index = site_crit.index.str.replace('sites.', '')
site_crit.index.name = 'column_name'


cols = site_crit.index
list(cols)
cols = list(cols)

site_crit

,citations,criterion,criterion_operation,criterion_value,description,table_column
column_name,,,,,,
dir_alpha95,This study,DE-SITE,<=,180,Criteria for selection of site direction,sites.dir_alpha95
dir_k,This study,DE-SITE,>=,50,Criteria for selection of site direction,sites.dir_k
dir_n_samples,This study,DE-SITE,>=,5,Criteria for selection of site direction,sites.dir_n_samples
dir_n_specimens_lines,This study,DE-SITE,>=,4,Criteria for selection of site direction,sites.dir_n_specimens_lines
int_abs_sigma,This study,IE-SITE,<=,-1,Criteria for selection of site intensity,sites.int_abs_sigma
int_abs_sigma_perc,This study,IE-SITE,<=,15,Criteria for selection of site intensity,sites.int_abs_sigma_perc
int_n_samples,This study,IE-SITE,>=,2,Criteria for selection of site intensity,sites.int_n_samples
site.dir_polarity,This study,NPOLE,=,n,Criteria for inclusion in normal mean,site.dir_polarity
site.dir_polarity,This study,RPOLE,=,r,Criteria for inclusion in reverse mean,site.dir_polarity


In [17]:
#sites.head().ix[cols]

In [18]:
# create string --> operator conversion    
import operator
ops = {"<": operator.lt, ">": operator.gt, "==": operator.eq, "<=": operator.le, ">=": operator.gt}

# function for applying criteria
def apply_crit(series, crit_series, criteria_type):
    """
    Apply 1 criterion (i.e., 1 row of the criteria table) to another table.
    Return series with boolean values for whether the row passes.
    """
    col_name = crit_series.name
    #print col_name
    # if there's no value, pass == True??  or == False?
    if not series[col_name]:
        return "{} not in row".format(col_name)
    # if we're missing criteria names, then what??
    elif not series['criteria']:
        return "no value in criteria"
    elif criteria_type not in series['criteria']:
        return "{} not in criteria for this row".format(criteria_type)
    crit_name = crit_series['criterion']
    crit_value = float(crit_series['criterion_value'])
    op_str = crit_series['criterion_operation']
    op = ops[op_str]
    value = float(series[col_name])
    #print value, op_str, crit_value
    result = op(value, crit_value)
    #print op
    #print result
    return result





In [19]:
# apply a criterion to a table
crit_name = 'dir_alpha95'
pass_name = 'pass_' + crit_name
crit1 = site_crit.ix[crit_name]
sites[pass_name] = sites.apply(apply_crit, axis=1, args=(crit1, 'DE-SITE'))
cond = sites[pass_name] == True
sites[cond][[crit_name, pass_name]].head()

,dir_alpha95,pass_dir_alpha95
site,,
mc01,4.2,True
mc02,2.1,True
mc03,2.3,True
mc04,4.6,True
mc06,4.8,True


In [20]:
for crit_name in site_crit.index[1:]:
    crit_series = site_crit.ix[crit_name]
    # if there are multiple records for a single crit_name, ignore that one
    if not isinstance(crit_series, pd.Series):
        continue
    sites['pass_' + crit_name] = sites.apply(apply_crit, axis=1, args=(crit_series, "DE-SITE"))

    
cond = sites.columns.str.contains('pass')
sites.head()
sites[sites.columns[cond]].head()

,pass_dir_alpha95,pass_dir_k,pass_dir_n_samples,pass_dir_n_specimens_lines,pass_int_abs_sigma,pass_int_abs_sigma_perc,pass_int_n_samples
site,,,,,,,
mc01,dir_alpha95 not in row,dir_k not in row,dir_n_samples not in row,dir_n_specimens_lines not in row,DE-SITE not in criteria for this row,DE-SITE not in criteria for this row,DE-SITE not in criteria for this row
mc01,DE-SITE not in criteria for this row,DE-SITE not in criteria for this row,DE-SITE not in criteria for this row,DE-SITE not in criteria for this row,int_abs_sigma not in row,int_abs_sigma_perc not in row,int_n_samples not in row
mc01,True,True,True,dir_n_specimens_lines not in row,int_abs_sigma not in row,int_abs_sigma_perc not in row,int_n_samples not in row
mc01,dir_alpha95 not in row,dir_k not in row,dir_n_samples not in row,dir_n_specimens_lines not in row,int_abs_sigma not in row,int_abs_sigma_perc not in row,int_n_samples not in row
mc02,dir_alpha95 not in row,dir_k not in row,dir_n_samples not in row,dir_n_specimens_lines not in row,int_abs_sigma not in row,int_abs_sigma_perc not in row,int_n_samples not in row


In [21]:
# create full McMurdo contribution

reload(nb)

con = nb.Contribution(wdir, custom_filenames={'specimens': 'specimens.txt', 'samples': 'samples.txt',
                                             'sites': 'sites.txt'})


criteria = con.tables['criteria'].df.copy()




def apply_crit(series, crit_series):#, criteria_type):
    """
    Apply 1 criterion (i.e., 1 row of the criteria table) to another table.
    Return series with boolean values for whether the row passes.
    """
    col_name = crit_series.name
    # if there's no value, pass == True
    if col_name not in series:
        return True
    if not series[col_name]:
        return True
        #return "{} not in row".format(col_name)
    # if there is a value, test that it is within correct limits
    crit_name = crit_series['criterion']
    crit_value = float(crit_series['criterion_value'])
    op_str = crit_series['criterion_operation']
    op = ops[op_str]
    value = float(series[col_name])
    result = op(value, crit_value)
    return result



def add_criteria_named(category_name, dtype):
    df = con.tables[dtype].df
    criteria_subset = criteria[criteria['criterion'] == category_name]
    criteria_subset.index = criteria_subset.index.str.replace(dtype + '.', '')
    pass_col_names = []
    for crit_name, crit_row in criteria_subset.iterrows():
        #print 'crit_name', crit_name
        col_name = category_name + "_" + crit_name + "_pass"
        #print 'col_name', col_name
        pass_col_names.append(col_name)
        df[col_name] = df.apply(apply_crit, args=(crit_row,), axis=1)
    return pass_col_names
    

#DE_SPEC = criteria[criteria['criterion'] == 'DE-SPEC']
#DE_SPEC.index = DE_SPEC.index.str.replace('specimens.', '')
#pass_col_names = []
#for crit_name, crit_row in DE_SPEC.iterrows():
#    #print 'crit_name', crit_name
#    col_name = 'DE-SPEC_' + crit_name + "_pass"
#    #print 'col_name', col_name
#    pass_col_names.append(col_name)
#    specimens[col_name] = specimens.apply(apply_crit, args=(crit_row,), axis=1)
    
    
dtype = 'specimens'
criteria_name = 'IE-SPEC'
pass_col = criteria_name + "_pass"
pass_col_names = add_criteria_named(criteria_name, dtype)
print pass_col_names
df = con.tables[dtype].df


col_names = df.columns[df.columns.str.contains(criteria_name)]
df[df[col_names].all(1)]#[col_names]


## all specimens that pass all DE-SPEC criteria
#df[df[pass_col]].index
#df.head()
df.head()[col_names]

['IE-SPEC_int_b_beta_pass', 'IE-SPEC_int_dang_pass', 'IE-SPEC_int_drats_pass', 'IE-SPEC_int_fvds_pass', 'IE-SPEC_int_mad_free_pass', 'IE-SPEC_int_maxdev_pass', 'IE-SPEC_int_n_ptrm_pass', 'IE-SPEC_int_q_pass']


,IE-SPEC_int_b_beta_pass,IE-SPEC_int_dang_pass,IE-SPEC_int_drats_pass,IE-SPEC_int_fvds_pass,IE-SPEC_int_mad_free_pass,IE-SPEC_int_maxdev_pass,IE-SPEC_int_n_ptrm_pass,IE-SPEC_int_q_pass
specimen,,,,,,,,
mc01a,True,True,True,True,True,True,True,True
mc01a,True,True,True,True,True,True,True,True
mc01b,True,True,True,True,True,True,True,True
mc01b,True,True,True,True,True,True,True,True
mc01c,True,True,True,True,True,True,True,True


In [22]:
reload(nb)
import pmagpy.pmag
reload(pmagpy.pmag)
con = nb.Contribution(wdir)#, custom_filenames={'specimens': 'custom_specimens.txt', 'samples': 'custom_samples.txt',
                                                                             #'sites': 'custom_sites.txt'})



con.tables

{'ages': <programs.new_builder.MagicDataFrame at 0x121fdea50>,
 'contribution': <programs.new_builder.MagicDataFrame at 0x1237eee50>,
 'criteria': <programs.new_builder.MagicDataFrame at 0x1204d39d0>,
 'images': <programs.new_builder.MagicDataFrame at 0x122028510>,
 'locations': <programs.new_builder.MagicDataFrame at 0x1237eef50>,
 'measurements': <programs.new_builder.MagicDataFrame at 0x103ea2150>,
 'samples': <programs.new_builder.MagicDataFrame at 0x12307be90>,
 'sites': <programs.new_builder.MagicDataFrame at 0x120747e50>,
 'specimens': <programs.new_builder.MagicDataFrame at 0x115179d50>}

In [23]:
# issues with anisotropy data from conversion script

print len(con.tables['specimens'].df[['aniso_tilt_correction']].dropna())

con.tables['specimens'].df[['aniso_s']].dropna()
print len(con.tables['specimens'].df[['aniso_v1', 'aniso_v2', 'aniso_v3']].dropna())
con.tables['specimens'].df[['aniso_s']].dropna()

38
19


,aniso_s
specimen,
mc121d1,0.001793
mc121f2,0.003433
mc121g1,0.004776
mc121k1,0.018612
mc121k2,0.015681
mc142c1,-0.027299
mc142d1,-0.005973
mc142e2,-0.035221
mc142f2,-0.028646


In [24]:
# troubleshooting dropping rows in dataframes -- non-unique index is a problem

site_df = con.tables['sites'].df
#thingee = set([0, 4])
#site_df.index[list(thingee)]

to_drop = [0, 4]
#site_df.drop(site_df.iloc[list(thingee)], inplace=True)
# this doesn't work, because it drops extra values with the same index value (mc01)
#site_df.drop(to_drop, inplace=True)

# this works
df = site_df.iloc[sorted(set(range(len(site_df))) - set([0, 4]))]

# this works
df = site_df.iloc[[i for i in range(len(site_df)) if i not in to_drop]]

# this works
site_df = site_df.reset_index(drop=True).drop(to_drop).set_index('site')
site_df.columns[site_df.columns.str.contains('age')]
site_df.columns[site_df.columns.str.contains("age($|_).*")]


/Users/nebula/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/ipykernel/__main__.py:21: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


Index([u'age', u'age_sigma', u'age_unit'], dtype='object')

## Translating 2.5 specimens files --> 3.0 specimen file

In [25]:
import SPD.mapping.map_magic as mm
reload(mm)
import programs.new_builder as nb
reload(nb)
wdir = os.path.join(os.getcwd(), "3_0", "McMurdo2.5")

# read in er_ data & make DataFrame
data1, dtype = pmag.magic_read(os.path.join(wdir, 'er_specimens.txt'))
d1 = pd.DataFrame(data1)
d1.index = d1['er_specimen_name']
# read in pmag_ data & make DataFrame
data2, dtype2 = pmag.magic_read(os.path.join(wdir, 'pmag_specimens.txt'))
d2 = pd.DataFrame(data2)
d2.index = d2['er_specimen_name']
# combine the two Dataframes
d3 = pd.concat([d1, d2])
# sort the DataFrame so that all records from one specimen are together
d3.sort_index(inplace=True)
# fix the column names 
col_names = d3.columns
#  change column names if needed
mapped_col_names = []
for col in col_names:
    if col in mm.spec_magic2_2_magic3_map:
        new_col_name = mm.spec_magic2_2_magic3_map[col]
        mapped_col_names.append(new_col_name)
    else:
        mapped_col_names.append(col)
    
# apply changed column names to DataFrame
d3.columns = mapped_col_names
d3.head()
# create a MagicDataFrame object, providing the dataframe and the data type
spec_df = nb.MagicDataFrame(dtype='specimens', df=d3)
# write out the data to file
spec_df.write_magic_file()


-I- writing specimens data to /Users/nebula/Python/PmagPy/specimens.txt


In [26]:
reload(nb)
con = nb.Contribution(os.path.join(os.getcwd(), '3_0', 'McMurdo'))

con.tables['ages'].df.head()

,age,age_sigma,age_unit,citations,description,location,method_codes,site
0,1.18,0.005,Ma,Tauxe et al. 2004,Excellent flat spectrum.,McMurdo,GM-ARAR-AP,mc01
1,0.33,0.01,Ma,Tauxe et al. 2004,"Reasonably flat, some recoil, climbs ar high T...",McMurdo,GM-ARAR-AP,mc02
2,0.348,0.004,Ma,Tauxe et al. 2004,"Excellent flat spectrum for 70%, then climbs a...",McMurdo,GM-ARAR-AP,mc03
3,0.34,0.003,Ma,Tauxe et al. 2004,Excellent flat spectrum with concordant WMPA a...,McMurdo,GM-ARAR-AP,mc04
4,0.65,0.025,Ma,Tauxe et al. 2004,Somewhat U-shaped spectrum with OK plateau in ...,McMurdo,GM-ARAR-AP,mc08


# Cleanup

In [27]:
!rm sites.txt samples.txt specimens.txt measurements.txt ages.txt contribution.txt images.txt criteria.txt locations.txt
!rm *.png
!rm *.jpg

rm: sites.txt: No such file or directory
rm: samples.txt: No such file or directory
rm: measurements.txt: No such file or directory
rm: ages.txt: No such file or directory
rm: contribution.txt: No such file or directory
rm: images.txt: No such file or directory
rm: criteria.txt: No such file or directory
rm: locations.txt: No such file or directory
rm: *.png: No such file or directory
rm: *.jpg: No such file or directory
